ex1 : cv score : 0.015197305813776704  
..........................................................................................................  
ex2:  
not scaling, remove vehicle, not use nonscored_target  : 0.015448837474049428  
not scaling, remove vehicle, not use nonscored_target, 7 seeds  : cv score : 0.015381427768256765  
not scaling, remove vehicle, not use nonscored_target, drop high corr cols  : 0.01545810170388885
not scaling, remove vehicle, not use nonscored_target, add 95% , 60% : 0.015533423204263394
not scaling, remove vehicle, not use nonscored_target, add 95% , 50% : 0.015430995160848253
not scaling, remove vehicle, not use nonscored_target, add 95% , 40% : 0.015354553654126092
all : 0.01515220913930692(7 seeds)
all : 0.015207047278156766 (10 seeds)
all : model tuning + 3 layer (7seeds) cv score : 0.014899280863407222 (LB : 0.1877)

混ぜるほど良いスコア。。。？  
high corr な特徴落としたら少し悪化
95%追加で少し改善、なんか全部やった方がいい説
..........................................................................................................  
ex3:
not scaling, remove vehicle, not use nonscored_target, use DAE(variance 0.5,　0.2776,　0.2227)  : 0.015192768673310545
not scaling, remove vehicle, not use nonscored_target, use DAE(variance 1.0,　0.3396,　0.2673)  : 0.015260472747032183 

not scaling, remove vehicle, not use nonscored_target, use DAE2(variance 0.5,　0.1886,　0.0274)  : 0.015196597745138379
not scaling, remove vehicle, not use nonscored_target, use DAE2(variance 1.0,　0.2707,　0.0456)  : 0.015279887596214419

all + DAE - drop : 0.014972256803365827
all + DAE - scale - drop : 0.015037600164910332
all + DAE : 0.014993220391337197

悪化はした
そこまで悪くはなさそう
cellの方がロスが落ちないから、チューニング次第な感じはする
ノイズのバリアンスを大きくするとロスは落ちにくくなる、0.5の時が良さそう
CVとLBの相関はある


In [ ]:
import warnings, random, os, sys
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import tensorflow as tf




pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline

def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def make_scaler(flag):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
seeds = [0, 1, 2, 3, 4, 5, 6]
SCALE = "quantile"
USE_DAE = True
USE_SCALE = True


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Concatenate, Embedding, Activation, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow_addons.layers import WeightNormalization
from tensorflow_addons.optimizers import AdamW, Lookahead


def DNN_3lmodel(input_size, output_size):
    inp = Input(shape = (input_size, ))
    x = BatchNormalization()(inp)
    x = Dropout(0.4914099166744246)(x)
    x = WeightNormalization(Dense(1159, activation = 'relu'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.18817607797795838)(x)
    x = WeightNormalization(Dense(960, activation = 'relu'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.12542057776853896)(x)
    x = WeightNormalization(Dense(1811, activation = 'relu'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.20175242230280122)(x)
    out = WeightNormalization(Dense(output_size, activation = 'sigmoid'))(x)
    model = Model(inputs = inp, outputs = out)
    opt = Adam(learning_rate = 0.001)
    opt = Lookahead(opt, sync_period = 10)
    model.compile(
        optimizer=opt, 
        loss=BinaryCrossentropy(label_smoothing = 0.0015),
    )
    return model

def DNN_model(input_size, output_size):
    inputs = Input((input_size, ))

    outputs = BatchNormalization()(inputs)
    outputs = Dropout(0.20)(outputs)
    outputs = WeightNormalization(Dense(1024, activation="relu"))(outputs)

    outputs = BatchNormalization()(outputs)
    outputs = Dropout(0.20)(outputs)
    outputs = WeightNormalization(Dense(1024, activation="relu"))(outputs)

    outputs = BatchNormalization()(outputs)
    outputs = WeightNormalization(Dense(output_size, activation="sigmoid"))(outputs)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(
        optimizer=AdamW(lr=0.001, weight_decay=1e-5, clipvalue=756), 
        loss=BinaryCrossentropy(label_smoothing = 0.0015)
    )

    return model

def DAE(input_size):
    emb_size = input_size//2
    inputs = Input((input_size, ))
    x = Dense(emb_size, activation="relu")(inputs)
    x = Dense(emb_size, activation="relu")(x)
    x = Dense(emb_size, activation="relu")(x)
    outputs = Dense(input_size, activation="linear")(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(optimizer='adam', loss='mse')

    return model

def DAE2(input_size):
    emb_size = 1500
    inputs = Input((input_size, ))
    x = Dense(emb_size, activation="relu")(inputs)
    x = Dense(emb_size, activation="relu")(x)
    x = Dense(emb_size, activation="relu")(x)
    outputs = Dense(input_size, activation="linear")(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(optimizer='adam', loss='mse')

    return model

In [ ]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../Data/Raw/test_features.csv")

y = pd.read_csv("../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
CELLS_50 = CELLS[:50]
BIOS = GENES + CELLS

SCORED_MOAS = [col for col in y.columns if col != "sig_id"]
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS

train_sigid = train_df["sig_id"]
test_sigid = test_df["sig_id"]

TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]

In [ ]:
# DAE
if USE_DAE:
    genes = pd.concat([train_df, test_df])[GENES].values
    cells = pd.concat([train_df, test_df])[CELLS].values
    
    factor = 0.5
    variance = 0.5
    gene_noise = np.random.normal(loc=0., scale=variance, size=genes.shape)
    cell_noise = np.random.normal(loc=0., scale=variance, size=cells.shape)

    noisy_genes = genes + factor*gene_noise
    noisy_cells = cells +  factor * cell_noise

    epochs = 300
    g_chck = ModelCheckpoint('keras_dae_g.h5', monitor='loss', save_best_only=True)
    c_chck = ModelCheckpoint('keras_dae_c.h5', monitor='loss', save_best_only=True)
    es = EarlyStopping(monitor='loss', patience=10, min_delta=0)

    g_model = DAE(genes.shape[1])
    c_model = DAE(cells.shape[1])

    #g_model.fit(noisy_genes, genes, batch_size=128, verbose=1, epochs=epochs, callbacks=[g_chck, es])
    #c_model.fit(noisy_cells, cells, batch_size=128, verbose=1, epochs=epochs, callbacks=[c_chck, es])

    g_model.load_weights('keras_dae_g.h5')
    c_model.load_weights('keras_dae_c.h5')

    trans_genes = g_model.predict(genes)
    trans_cells = c_model.predict(cells)
    print(trans_genes.shape)
    print(trans_cells.shape)
    
    train_trans_genes = trans_genes[:TR_SIZE]
    test_trans_genes = trans_genes[TR_SIZE:]
    train_trans_cells = trans_cells[:TR_SIZE]
    test_trans_cells = trans_cells[TR_SIZE:]
    for i, col in enumerate(GENES):
        train_df[col] = train_trans_genes[:,i]
        test_df[col] = test_trans_genes[:,i]
    for i, col in enumerate(CELLS):
        train_df[col] = train_trans_cells[:,i]
        test_df[col] = test_trans_cells[:,i]


In [ ]:
train_df["c-0"].hist(bins=100)

In [ ]:
#
drop_cols = [
    'g-37', 'g-38', 'g-50', 'g-63', 'g-121', 'g-123', 'g-195', 'g-228', 'g-248', 'g-261', 'g-328', 'g-349', 
    'g-369', 'g-460', 'c-1', 'c-2', 'c-4', 'c-6', 'c-8', 'c-10', 'c-11', 'c-13', 'c-17', 'c-18', 'c-26', 'c-31', 'c-33', 
    'c-38', 'c-40', 'c-42', 'c-51', 'c-52', 'c-55', 'c-60', 'c-66', 'c-73', 'c-75'
]
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)

BIOS = list(set(BIOS) - set(drop_cols))

In [ ]:
# 片側95%に入ったときに50%以上の確率で判別できるMoAの組み合わせが存在した列
n_cols = ['g-270', 'g-170', 'g-718', 'g-163', 'g-377', 'g-712', 'g-529', 'g-178', 'g-251', 'g-74', 'g-424', 'g-330', 'g-97', 'g-417', 'g-144', 'g-689', 'g-215', 'g-317', 'g-201', 'g-731', 'g-300', 'g-500', 'g-368', 'g-578', 'g-468', 'g-338', 'g-296', 'g-157', 'g-116', 'g-478', 'g-629', 'g-100', 'g-65', 'g-243', 'g-53', 'g-443', 'g-226', 'g-46', 'g-158', 'g-389', 'g-577', 'g-147', 'g-90', 'g-131', 'g-745', 'g-750', 'g-619', 'g-646', 'g-12', 'g-130', 'g-392', 'g-106', 'g-419', 'g-768', 'g-386', 'g-596', 'g-31', 'g-546', 'g-600', 'g-541', 'g-512', 'g-723', 'g-764', 'g-208', 'g-467', 'g-264', 'g-148', 'g-86', 'g-744', 'g-266', 'g-91', 'g-146', 'g-102']
p_cols = ['c-16', 'c-57', 'g-726', 'c-53', 'g-423', 'c-97', 'g-624', 'g-178', 'g-257', 'c-29', 'c-50', 'g-128', 'c-63', 'g-674', 'c-99', 'c-43', 'c-46', 'c-72', 'c-20', 'c-39', 'g-439', 'g-390', 'g-540', 'c-64', 'c-47', 'c-89', 'g-497', 'c-95', 'g-55', 'g-293', 'g-643', 'g-759', 'g-590', 'g-411', 'g-508', 'c-71', 'g-683', 'c-81', 'g-534', 'g-503', 'g-298', 'g-760', 'g-231', 'g-300', 'c-82', 'c-22', 'g-276', 'c-92', 'g-306', 'c-86', 'c-59', 'c-88', 'c-3', 'g-250', 'g-67', 'g-428', 'g-691', 'c-19', 'g-489', 'g-667', 'c-34', 'c-54', 'c-27', 'c-44', 'g-385', 'c-62', 'g-769', 'g-704', 'g-353', 'c-68', 'c-84', 'c-7', 'g-332', 'c-28', 'g-735', 'c-98', 'c-12', 'c-78', 'c-24', 'c-79', 'g-558', 'g-491', 'c-65', 'g-644', 'g-370', 'c-48', 'c-41', 'c-56', 'c-30', 'c-85', 'c-21', 'g-40', 'c-94', 'c-49', 'c-35', 'c-96', 'c-5', 'c-61', 'g-743', 'g-62', 'g-8', 'c-93', 'c-37', 'c-70', 'g-705', 'g-438', 'c-0', 'c-36', 'c-90', 'g-66', 'g-185', 'g-421', 'g-291', 'g-321', 'c-80', 'g-75', 'g-202', 'g-685', 'c-67', 'c-83', 'g-161', 'c-15', 'g-568', 'c-32', 'g-375', 'g-513', 'g-72', 'g-58', 'c-87', 'c-25', 'g-477', 'g-406', 'c-14', 'c-77', 'g-113', 'g-588', 'g-672', 'g-112', 'g-204', 'c-91', 'c-45']
for col in n_cols:
    train_df["n_"+col] = train_df[col].map(lambda x : 1 if x <= -9.5 else 0)
    test_df["n_"+col] = test_df[col].map(lambda x : 1 if x <= -9.5 else 0)
    
for col in p_cols:
    train_df["p_"+col] = train_df[col].map(lambda x : 1 if x >= 9.5 else 0)
    test_df["p_"+col] = test_df[col].map(lambda x : 1 if x >= 9.5 else 0)

In [ ]:
if USE_SCALE:
    for val in train_df["time_dose"].unique():
        temp = pd.concat([train_df[train_df["time_dose"] == val], test_df[test_df["time_dose"] == val]])
        scaler = make_scaler(SCALE)
        scaler.fit(temp[BIOS].values.reshape(-1, len(BIOS)))
        tr_ind = train_df[train_df["time_dose"] == val].index
        te_ind = test_df[test_df["time_dose"] == val].index
        tr_trans = scaler.transform(train_df[BIOS].iloc[tr_ind].values.reshape(-1, len(BIOS))).T
        te_trans = scaler.transform(test_df[BIOS].iloc[te_ind].values.reshape(-1, len(BIOS))).T

        for i, col in enumerate(BIOS):
            train_df[col].iloc[tr_ind] = tr_trans[i]
            test_df[col].iloc[te_ind] = te_trans[i]

drop_cols = ["cp_time", "cp_dose", "time_dose"]

train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot"), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot"), test_df.drop(drop_cols, axis=1) ], axis=1)




# remove cp_type = ctl_vehicle
print(train_df.shape)
print(test_df.shape)
print()

mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)

In [ ]:
X = train_df.drop("sig_id", axis=1)
test_X = test_df.drop("sig_id", axis=1)
y_nonv = y[mask].drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

print(X.shape)
print(test_X.shape)
print(y_nonv.shape)

In [ ]:
train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_X.shape[0], y_nonv.shape[1]))

for seed in seeds:
    seed_everything(seed)
    K = 7
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    for itr, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):
        print("======================== fold {} ========================".format(itr+1))
        train_X = X.iloc[train_index].values
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index].values
        valid_y = y_nonv[valid_index]
        
        model = DNN_3lmodel(input_size=train_X.shape[1], output_size=train_y.shape[1])
        #model = DNN_model(input_size=train_X.shape[1], output_size=train_y.shape[1])
        cb = EarlyStopping(monitor='val_loss', patience=3)
        
        early_stopping = EarlyStopping(monitor='val_loss',
                                                          mode = 'min',
                                                          patience = 10,
                                                          restore_best_weights = False,)
        
        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                                         mode = 'min',
                                                         factor = 0.3,
                                                         patience = 3,)
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint(f'_{itr}_{seed}.h5',
                                                        monitor='val_loss',
                                                        save_best_only = True,
                                                        save_weights_only = True)
        
        
        model.fit(
            train_X, 
            train_y,
            batch_size=128,
            epochs=80,
            verbose=0,
            #callbacks=[cb],
            callbacks = [early_stopping, reduce_lr,  checkpoint],
            validation_data=(valid_X, valid_y),
        )
        #model.load_weights(f'_{itr}_{seed}.h5')
        
        train_pred[valid_index] += model.predict(valid_X, batch_size=128)
        preds += model.predict(test_X.values, batch_size=128) / (K*len(seeds))


    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

In [ ]:
train_preds2 = np.zeros((TR_SIZE, y.shape[1]))
preds2 = np.zeros((TE_SIZE, y.shape[1]))

train_preds2[train_nonvehicle_index] = train_preds
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
p = 0.001
t = 0
def func(t, p):
    
    return -(t*np.log(p) + (1-t) * np.log(1-p))
print(func(y[:,0], train_preds2[:,0]))

In [ ]:
train_preds2[-2,0]

In [ ]:
y[-2,0]

In [ ]:
1*np.log(0.5)

In [ ]:
0*np.log(1e-10)

In [ ]:
preds = [:,:tag_size]
sub_df = pd.read_csv("../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
#sub_df.to_csv("submission.csv", index=False)

In [ ]:
t = train_preds.copy()
print(metric(y[:,:tag_size], t[:,:tag_size]))
t_ = train_df[train_df["cp_type"] == 0]
t[t_.index] = np.zeros((t_.shape[0], t.shape[1]))
t = np.where(t > 1, 1, t)
print(metric(y[:,:tag_size], t[:,:tag_size]))

In [ ]:
li = []
not_li = []
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        if y[i][j] == 1:
            #print("====={}, {}====".format(i,j))
            rank = np.where(train_preds[i].argsort()[::-1] == j)[0][0]+1
            #print("rank {}".format(rank))
            if rank <= 20:
                li.append(j)
            else:
                not_li.append(j)
            #print(train_preds[i][j])

In [ ]:
li = list(set(li))
not_li = list(set(not_li))
for i in li:
    if i not in not_li:
        print(i)

In [ ]:
cols = pd.read_csv("../Data/Raw/train_targets_nonscored.csv").columns
pd.read_csv("../Data/Raw/train_targets_nonscored.csv")[cols[256-206]]